In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Activation, Flatten
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50

print("Import Keras")

Using TensorFlow backend.


Import Keras


In [3]:
### Specify Training Parameters
batchSize = 64               #-- Training Batch Size
num_classes = 13             #-- Number of classes
num_epochs = 35              #-- Number of epochs for training   
learningRate= 0.001          #-- Learning rate for the network
lr_weight_decay = 0.95       #-- Learning weight decay. Reduce the learn rate by 0.95 after epoch

In [4]:
model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = model.output
x = GlobalAveragePooling2D()(x)
#x = Dense(1024, activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
#x = Dense(1024, activation='relu')(x) #dense layer 2
#x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x) #dense layer 3
#x = Dense(256, activation='relu')(x) #dense layer 4
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x) #dense layer 5
x = Dropout(0.5)(x)
#x = Dense(64, activation='relu')(x) #dense layer 6
#x = Dense(32, activation='relu')(x)
preds = Dense(num_classes, activation='softmax')(x) #final layer with softmax activation

model2 = Model(inputs=model.input, outputs=preds)
for layer in model2.layers[:20]:
    layer.trainable=False
for layer in model2.layers[20:]:
    layer.trainable=True
    
model2.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [5]:
model3 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x2 = model3.output
x2 = GlobalAveragePooling2D()(x2)
#x = Dense(1024, activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
#x = Dense(1024, activation='relu')(x) #dense layer 2
#x = Dropout(0.5)(x)
x2 = Dense(512, activation='relu')(x2) #dense layer 3
#x = Dense(256, activation='relu')(x) #dense layer 4
x2 = Dropout(0.5)(x2)
x2 = Dense(128, activation='relu')(x2) #dense layer 5
x2 = Dropout(0.5)(x2)
#x = Dense(64, activation='relu')(x) #dense layer 6
#x = Dense(32, activation='relu')(x)
preds2 = Dense(num_classes, activation='softmax')(x2) #final layer with softmax activation

model4 = Model(inputs=model3.input, outputs=preds2)
for layer in model4.layers[:175]:
    layer.trainable=False
for layer in model4.layers[175:]:
    layer.trainable=True
    
model4.summary()

/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________

In [6]:
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input, zoom_range=0.1, horizontal_flip=True) #, validation_split=0.13) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/', # this is where you specify the path to the main data folder
                                                 target_size=(224, 224),
                                                 batch_size=batchSize,
                                                 class_mode='categorical',
                                                 shuffle=True) #,
#                                                 subset="training")

#validation_generator=train_datagen.flow_from_directory('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/', # this is where you specify the path to the main data folder
#                                                 target_size=(224, 224),
#                                                 batch_size=batchSize,
#                                                 class_mode='categorical',
#                                                 shuffle=False,
#                                                 subset="validation")

Found 2819 images belonging to 13 classes.


In [7]:
#sgd = SGD(lr=learningRate, decay=lr_weight_decay)
adam = Adam(lr=learningRate, decay=lr_weight_decay)

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
#step_size_validation=validation_generator.n//validation_generator.batch_size
model2.fit_generator(generator=train_generator, 
                     steps_per_epoch=step_size_train, 
#                     validation_data=validation_generator, 
#                     validation_steps=step_size_validation, 
                     epochs=num_epochs)

Epoch 1/35
44/44 [==============================] - 37s 840ms/step - loss: 2.4550 - accuracy: 0.3637
Epoch 2/35
44/44 [==============================] - 30s 691ms/step - loss: 0.9916 - accuracy: 0.6581
Epoch 3/35
44/44 [==============================] - 30s 691ms/step - loss: 0.7233 - accuracy: 0.7564
Epoch 4/35
44/44 [==============================] - 31s 703ms/step - loss: 0.6334 - accuracy: 0.7808
Epoch 5/35
44/44 [==============================] - 31s 698ms/step - loss: 0.5253 - accuracy: 0.8174
Epoch 6/35
44/44 [==============================] - 31s 696ms/step - loss: 0.5273 - accuracy: 0.8374
Epoch 7/35
44/44 [==============================] - 31s 705ms/step - loss: 0.4981 - accuracy: 0.8457
Epoch 8/35
44/44 [==============================] - 31s 698ms/step - loss: 0.4219 - accuracy: 0.8672
Epoch 9/35
44/44 [==============================] - 31s 699ms/step - loss: 0.4288 - accuracy: 0.8624
Epoch 10/35
44/44 [==============================] - 32s 723ms/step - loss: 0.3620 - accura

In [8]:
adam = Adam(lr=learningRate, decay=lr_weight_decay)

model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
#step_size_validation=validation_generator.n//validation_generator.batch_size
model4.fit_generator(generator=train_generator, 
                     steps_per_epoch=step_size_train, 
#                     validation_data=validation_generator, 
#                     validation_steps=step_size_validation, 
                     epochs=num_epochs)

Epoch 1/35
44/44 [==============================] - 32s 735ms/step - loss: 1.7774 - accuracy: 0.4203
Epoch 2/35
44/44 [==============================] - 32s 723ms/step - loss: 0.9138 - accuracy: 0.6922
Epoch 3/35
44/44 [==============================] - 33s 745ms/step - loss: 0.6433 - accuracy: 0.7802
Epoch 4/35
44/44 [==============================] - 31s 711ms/step - loss: 0.5336 - accuracy: 0.8166
Epoch 5/35
44/44 [==============================] - 33s 743ms/step - loss: 0.5444 - accuracy: 0.8138
Epoch 6/35
44/44 [==============================] - 32s 732ms/step - loss: 0.4558 - accuracy: 0.8472
Epoch 7/35
44/44 [==============================] - 32s 727ms/step - loss: 0.3853 - accuracy: 0.8711
Epoch 8/35
44/44 [==============================] - 32s 739ms/step - loss: 0.3612 - accuracy: 0.8860
Epoch 9/35
44/44 [==============================] - 32s 729ms/step - loss: 0.3761 - accuracy: 0.8813
Epoch 10/35
44/44 [==============================] - 33s 754ms/step - loss: 0.3132 - accura

In [9]:
import cv2

test_data_name = []
test_data = []

for dirname, _, filenames in os.walk('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test'):
    for filename in filenames:
        test_data_name.append(filename)
        image = cv2.imread(os.path.join(dirname, filename))
        test_data.append(image)
        
print("Number of test data =", len(test_data))

Number of test data = 1040


In [10]:
def processing(image_data):
    #img = [data[123], data[1054], data[156]]
    #print("Original size:", img[1].shape)
    #print(training_label[1054])
    #original = img[1]
    #display_one(original)
    
    height = 224
    width = 224
    dim = (width, height)
    res_img = []
    
    for i in range(len(image_data)):
        res = cv2.resize(image_data[i], dim, interpolation=cv2.INTER_LINEAR)
        res_img.append(res)
    
    return res_img

test_data_reshape = processing(test_data)

print(test_data_reshape[0].shape)

(224, 224, 3)


In [11]:
for i in range(len(test_data_reshape)):
    test_data_reshape[i] = preprocess_input(test_data_reshape[i])

In [12]:
#for i in range(len(test_data_reshape)):
#    test_data_reshape[i] = (test_data_reshape[i] - 128.0) / 128.0
   
for i in range(len(test_data_reshape)):
    test_data_reshape[i] = test_data_reshape[i].reshape(-1, 224, 224, 3)
    
scene_detail = ["bedroom", "coast", "forest", "highway", "insidecity", "kitchen", "livingroom", "mountain", "office", "opencountry", "street", "suburb", "tallbuilding"]

print("id,label")

for i in range(len(test_data_reshape)):
    name_arr = test_data_name[i].split('.')
    name = name_arr[0]
    answer2 = model2.predict(test_data_reshape[i])
    answer4 = model4.predict(test_data_reshape[i])
    answer = [answer2[i] * answer4[i] for i in range(len(answer2))] 
    print(name + "," + scene_detail[np.argmax(answer)])


id,label
image_0547,tallbuilding
image_0334,office
image_0427,insidecity
image_0705,opencountry
image_0115,street
image_0275,insidecity
image_0653,bedroom
image_0451,tallbuilding
image_0443,street
image_0447,insidecity
image_0291,bedroom
image_0878,forest
image_0888,forest
image_0468,street
image_0718,coast
image_0902,suburb
image_0661,office
image_0932,opencountry
image_0518,suburb
image_0296,tallbuilding
image_0763,suburb
image_0356,mountain
image_0823,tallbuilding
image_0208,suburb
image_0681,opencountry
image_0128,suburb
image_0896,forest
image_0255,highway
image_0638,highway
image_0626,kitchen
image_0977,highway
image_0604,suburb
image_0332,livingroom
image_0181,street
image_0839,opencountry
image_0116,livingroom
image_0043,livingroom
image_0183,livingroom
image_0516,office
image_0995,livingroom
image_0197,kitchen
image_0325,tallbuilding
image_0667,opencountry
image_0868,office
image_0965,mountain
image_0003,bedroom
image_0060,forest
image_0105,mountain
image_0319,highway
image_06

image_0212,mountain
image_0786,livingroom
image_0016,highway
image_0136,tallbuilding
image_0402,opencountry
image_0196,livingroom
image_0292,opencountry
image_0294,tallbuilding
image_0237,opencountry
image_0118,street
image_0749,opencountry
image_0708,opencountry
image_0611,bedroom
image_0011,livingroom
image_0620,street
image_0631,kitchen
image_0643,tallbuilding
image_0090,suburb
image_0077,coast
image_0031,forest
image_0592,forest
image_0837,forest
image_0537,office
image_0550,street
image_0409,suburb
image_0783,insidecity
image_0968,mountain
image_0522,street
image_0139,forest
image_0704,livingroom
image_0870,highway
image_0353,opencountry
image_0331,mountain
image_0682,kitchen
image_0742,suburb
image_0536,bedroom
image_0428,tallbuilding
image_0482,bedroom
image_1016,livingroom
image_0130,livingroom
image_0327,forest
image_0252,suburb
image_0180,tallbuilding
image_0477,mountain
image_0288,highway
image_1025,insidecity
image_0826,suburb
image_0432,coast
image_0714,forest
image_0706,o

image_0897,opencountry
image_0597,office
image_0753,highway
image_0396,mountain
image_0095,livingroom
image_0111,forest
image_0281,coast
image_0126,forest
image_0721,street
image_0849,insidecity
image_0262,tallbuilding
image_0166,suburb
image_0064,coast
image_0315,suburb
image_0036,street
image_0603,opencountry
image_1019,coast
image_0302,coast
image_0486,insidecity
image_0744,insidecity
image_0330,livingroom
image_0191,mountain
image_0133,insidecity
image_0260,insidecity
image_0555,bedroom
image_0457,suburb
image_0073,livingroom
image_0229,suburb
image_0215,forest
image_0648,livingroom
image_0056,tallbuilding
image_0479,suburb
image_0230,office
image_0813,street
image_0619,coast
image_0777,coast
image_0014,insidecity
image_0406,opencountry
image_0295,opencountry
image_1020,highway
image_0981,opencountry
image_0601,office
image_0675,bedroom
image_0983,office
image_0670,livingroom
image_0389,office
image_0032,livingroom
image_0329,mountain
image_0321,opencountry
image_0429,office
image_